In [1]:
#импортируем необходимые библиотеки
import requests
from bs4 import BeautifulSoup
import time
from nltk import word_tokenize
import json
import telebot
import heapq
from telebot import TeleBot, types
import spacy
from nltk.corpus import stopwords
stopwords = stopwords.words('russian')
my_stop_words = {'ка', 'весь', 'свой', 'это', 'лишь'}
stopwords = my_stop_words.union(stopwords)
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from ruaccent import load_accentor
accentor = load_accentor()
from hybrid_vectorizer import HybridVectorizer
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from sklearn.feature_extraction.text import TfidfVectorizer
nlp = spacy.load("ru_core_news_md")

Loading Custom Accentor
✅ Using CPU
📖 Loading vocabulary from C:\Users\йцукен\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ruaccent\model\vocab.json...
   Vocabulary size: 224
🤖 Initializing model...
📦 Loading weights from C:\Users\йцукен\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ruaccent\model\acc_model.pt...
✅ Accentor initialized successfully!



C:\Users\йцукен\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-02-24 11:00:41,373 [INFO] pymorphy3.opencorpora_dict.wrapper - Loading dictionaries from C:\Users\йцукен\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymorphy3_dicts_ru\data
2026-02-24 11:00:41,788 [INFO] pymorphy3.opencorpora_dict.wrapper - format: 2.4, revision: 417150, updated: 2022-01-08T22:09:24.565962


Парсинг имён авторов, названий и текстов стихотворений

In [ ]:
#получаем ссылки на страницы сайта
def page_generator (x,y):
    page_links = []
    for i in range(x, y):
        page_links.append(f'https://www.culture.ru/literature/poems?page={i}')
    return (page_links)

#получаем ссылки на стихотворения
def link_generator (page_links):
    poem_links = []
    poem_elements = []
    for i in page_links:
        page = requests.get(i)
        if page.status_code != 200:
            print ('ошибка подключения к странице')
            break
        time.sleep(3)
        soup = BeautifulSoup(page.text)
        poem_elements = soup.find_all('a', class_='styles_PoemCard__Heading__SJxDJ')
        for element in poem_elements:
            href = element.get('href')
            full_url = ['https://www.culture.ru' + href]
            poem_links.extend(full_url)
    return (poem_links)

#создаём функцию для поиска текста стихотворения на странице
def poem_finder (soup):
    poem = soup.find (class_='styles_body__WEo9w')
    poem = str(poem)
    poem = poem.replace ('<br/>', '\n')
    poem = BeautifulSoup(poem, 'html.parser').text
    return (poem)

#создаём функцию для поиска имени автора
def author_finder (soup):
    author = soup.find (class_='styles_MainPoem__Author__SjIRK').text
    return (author)

#создаём функцию для поиска и токенизации названия стиховторения
def title_finder (soup):
    title = soup.find (class_='styles_MainPoem__Title__nvD6h').text
    title = word_tokenize (title)
    return (title)

#находим имя автора, название и текст каждого стихотворения
author_list = []
title_list = []
poem_list = []
ranges = [(1,5), (5,10), (10,15), (15,21)]
for x, y in ranges:
    page_links = page_generator (x,y)
    print ('ссылки на страницы:', page_links)
    poem_links = link_generator (page_links)
    print ('ссылки на стихотворения:', poem_links)
    time.sleep(3)
    for link in poem_links:
        page = requests.get(link)
        if page.status_code != 200:
            print ('ошибка подключения к странице')
            page = requests.get(link)
            if page.status_code != 200:
                print ('повторная ошибка подключения к странице')
                continue
        time.sleep(3)
        soup = BeautifulSoup(page.text)
        author = author_finder (soup)
        title = title_finder (soup)
        poem = poem_finder (soup)
        author_list.append (author)
        title_list.append (title)
        poem_list.append (poem)
        print (title)

In [ ]:
#сохраняем информацию в файле
#data = {
    'title_list': [title_list],
    'author_list': [author_list],
    'poem_list': poem_list
}
#with open('poem_data.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

Обработка текстов перед кластеризацией

Тематическое распределение текстов

In [ ]:
#лемматизация текстов стихотворений
with open ('poem_data.json', 'r', encoding='utf-8') as file: #открываем созданный ранее файл с данными, полученными в ходе парсинга
    datum = json.load(file)
lemmas_list = []
for i in datum['poem_list']:
    doc = re.sub(r'[^а-яёА-ЯЁa-zA-Z]', ' ', i)
    doc = nlp(doc)
    lemmas = [token.lemma_ for token in doc if not token.is_space]
    lemmas_list.append (lemmas)

In [7]:
with open ('poem_data.json', 'r', encoding='utf-8') as file: #открываем созданный ранее файл с данными, полученными в ходе парсинга
    datum = json.load(file)

In [ ]:
#сохранение файла с лемматизированными текстами
data = {'lemmas_list': lemmas_list}
with open('lemmas_data.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

In [ ]:
with open ('lemmas_data.json', 'r', encoding='utf-8') as file: #открываем созданный ранее файл с данными, полученными в ходе парсинга
    datum2 = json.load(file)
lemmas = datum2['lemmas_list']
texts = [' '.join(poem) for poem in lemmas]
stopwords_list = list (stopwords)
#делим тексты на тематически схожие группы
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# подготовка данных
vectorizer = CountVectorizer(max_features=1000, stop_words=stopwords_list)
X = vectorizer.fit_transform(texts)

# применение LDA
num_topics = 15
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

# вычисляем распределение тем по стихотворениям
doc_topic_dist = lda.transform(X)
dominant_topics = np.argmax(doc_topic_dist, axis=1)

#можно вывести и посмотреть наборы популярных в каждой теме слов
#for topic_idx, topic_words in enumerate(lda.components_):
#    top_words_idx = topic_words.argsort()[-10:][::-1]
#    top_words = [vectorizer.get_feature_names_out()[i] for i in top_words_idx]
#    print(f"Тема {topic_idx + 1}: {', '.join(top_words)}")

In [ ]:
#поиск наиболее часто встречающихся слов с целью фильтрации неинформативных единиц
texts = [' '.join(poem) for poem in lemmas] #делаем список удобоваримым для векторайзера
vect = TfidfVectorizer (stop_words=list(stopwords), max_features=32000)
dataset = vect.fit_transform(texts) #векторизация слов, вычисление частотности
idf_values = vect.idf_ 
feature_names = vect.get_feature_names_out()
df_idf = pd.DataFrame({'слово': feature_names, 'idf': idf_values})
df_sorted_asc = df_idf.sort_values('idf', ascending=True)
print(df_sorted_asc.head(50)) #просмотр наиболее популярных единиц - неинйормативные были добавлены в стоп лист

In [ ]:
#подбор оптимального колическтва тем для LDA при помощи рассчёта перплексии и когерентности
perplexity_scores = []
coherence_scores = []
min_topics = 2
max_topics = 25
step = 1
topic_range = range(min_topics, max_topics + 1, step)

#рассчёт перплексии
texts_for_sklearn = [' '.join(poem) for poem in lemmas]
vectorizer = CountVectorizer(max_features=1000, stop_words=stopwords_list) #создание матрицы со словами и частотностями их употребления
X = vectorizer.fit_transform(texts_for_sklearn)
lda_sklearn = LatentDirichletAllocation(n_components=k, #настройка LDA
                                        random_state=42,
                                        max_iter=100,
                                        learning_method='online')
lda_sklearn.fit(X) #создание модели LDA
perplexity = lda_sklearn.perplexity(X) #рассчёт перплексии
perplexity_scores.append(perplexity)
plot1.plot(topic_range, perplexity_scores, marker='o')
plot1.set_xlabel('Темы')
plot1.set_ylabel('Перплексия')
plot1.grid(True)

#рассчёт когерентности
texts_for_gensim = lemmas
id2word = dictionary(texts_for_gensim)
corpus = [id2word.doc2bow(text) for text in texts_for_gensim]
lda_gensim = LdaModel(corpus=corpus,
                      id2word=id2word,
                      num_topics=k,
                      random_state=42,
                      passes=10, 
                      alpha='auto')
coherence_model = CoherenceModel(model=lda_gensim, #нстройка модели когерентности
                                  texts=texts_for_gensim,
                                  dictionary=id2word,
                                  coherence='c_v')  
coherence = coherence_model.get_coherence() #создание модели когерентности
coherence_scores.append(coherence)
plot2.plot(topic_range, coherence_scores, marker='s', color='orange')
plot2.set_xlabel('Темы')
plot2.set_ylabel('Когерентность')
plot2.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
#сохраняем индексы стихотворений и их темы в файл
with open ('poem_data.json', 'r', encoding='utf-8') as file: #открываем созданный ранее файл с данными, полученными в ходе парсинга
    datum = json.load(file)
poem_indices = list(range(len(datum['poem_list'])))
df = pd.DataFrame({
    'стихотворение': poem_indices,          # или реальные названия
    'тема': dominant_topics
})
df.to_csv('poem_table.csv', index=False, encoding='utf-8')

Определение видов ритма и рифмы в текстах

In [ ]:
with open ('poem_data.json', 'r', encoding='utf-8') as file: #открываем созданный ранее файл с данными, полученными в ходе парсинга
    datum = json.load(file)

#определение видов ритма в текстах

def parting (text): #создаём функцию для разбиения текстов стихотворений на строки
    parted = text.lower()
    parted = re.sub(r'\xa0', ' ', parted) #убираем некрасивую штуку
    parted = re.sub(r'\n', 'РРР', parted) #используем условный набор символов, чтобы разделить тексты
    parted = [part.strip() for part in parted.split("РРР") if part.strip()]
    return (parted)  

def accentizing (text): #расставляем ударения в словах строк
    parted = parting (text)
    accentized = [accentor(s) for s in parted[:10]]
    accentized = [re.sub(r'[Ёё]', 'ё\'', s) for s in accentized] #ставим ударения над ё, получаем текст с полностью проставленными ударениями
    accentized = [encoding_vowels(s) for s in accentized]
    return (accentized)

def encoding_vowels(accentized): #делаем из текстовых строк последовательности ударных и безударных гласных, где 1 - ударные, 0 - безударные
    encoded = re.sub(r'[^аеёиоуыэюя\']', '', accentized) #заменяем 
    encoded = re.sub(r'[(аеёиоуыэюя)]\'', '1', encoded)
    encoded = re.sub(r'[(аеёиоуыэюя)]', '0', encoded)
    return (encoded)

def rhythm_counter(accentized): #определяем размер стихотворения и условно обозначаем каждый числом от 0 до 4
    accentized = accentizing (accentized)
    chunks = [[s[i:i+2] for i in range(0, len(s), 2)] for s in accentized] #делим последовательности цифр на последовательности по две цифры в каждой, чтобы проверить ямб и хорей
    iambus = 0
    choreus = 0
    for chunk in chunks:
        for element in chunk:
            if element == '01':
                iambus += 1
            elif element == '10':
                choreus += 1
    num_chunks = sum(len(sublist) for sublist in chunks)
    iambus_score = iambus/num_chunks #находим долю популярности ямба
    choreus_score = choreus/num_chunks #находим долю популярности хорея
    chunks = [[s[i:i+3] for i in range(0, len(s), 3)] for s in accentized] #делим последовательности цифр на последовательности по три цифры в каждой, чтобы проверить дактиль, амфибрахий и анапест
    dactyl = 0
    anapest = 0
    amphibrachium = 0
    for chunk in chunks:
        for element in chunk:
            if element == '100':
                dactyl += 1
            elif element == '010':
                amphibrachium += 1
            elif element == '001': 
                anapest += 1
    num_chunks = sum(len(sublist) for sublist in chunks)
    dactyl_score = dactyl/num_chunks #находим долю популярности дактиля
    amphibrachium_score = amphibrachium/num_chunks #находим долю популярности амфибрахия
    anapest_score = anapest/num_chunks #находим долю популярности анапеста
    score = max (iambus_score, choreus_score, dactyl_score, amphibrachium_score, anapest_score)
    scores = [choreus_score, iambus_score, dactyl_score, amphibrachium_score, anapest_score]
    score = scores.index(max(scores)) #выводим размер ввиде условного числа
    print (score)
    return (score)

score_line = [] #создаём список размеров всех текстов

for i in datum['poem_list']:    
    score_line.append(rhythm_counter(i))

#определение видов рифмы

def rhyme_counter (parted):
    parted = parting(parted) #делим стихотворения на строки
    parted = (re.sub(r'[^\w\s]', '', i) for i in parted) #убираем лишние символы из строк
    endings = []
    endings = [i[-2:] for i in parted] #оставляем только окончания последний слов в строках (вроде "ая", "им" и т.д.)
    ends1 = []
    ends2 = []
    for i in range(0, len(endings), 4): #создаём списки окончаний соседних строк (1 - 2 и т.д.)
        ends1.extend(endings[i:i+2])
        ends2.extend(endings[i+2:i+4])
    paired_rhyme = pattern_similarity (ends1, ends2) #рассчитываем популярность парной рифмы
    if paired_rhyme == 10: 
        return (2) #выводим обозначение для прозаических текстов
    ends1 = endings[0::2] #создаём списки окончаний в строках через одну (1 - 3 и т.д.)
    ends2  = endings[1::2]
    in_one_rhyme = pattern_similarity (ends1, ends2) #рассчитываем популярность перекрёстной рифмы
    rhyme_score = max (paired_rhyme, in_one_rhyme) 
    if rhyme_score == paired_rhyme:
        rhyme_score = 0
    else:
        rhyme_score = 1
    print (rhyme_score)
    return (rhyme_score)

def pattern_similarity(ends1, ends2): #создаём функцию для поиска соответствий в двух списках окончаний
    min_len = min(len(ends1), len(ends2)) #сокращаем списки до длины кратчайшего из них, чтобы не возникало несоответствий
    if min_len == 0:
        return (10) #для стихотворений, состоящих из одной строки, создаём особое обозначение. таких стихотворений в базе два, они оба написаны в прозе
    matches = sum(1 for i in range(min_len) if ends1[i] == ends2[i])
    return (matches / min_len)

rhyme_line = [] #создаём список видов рифм текстов
for i in datum['poem_list']:
    rhyme_line.append(rhyme_counter(i))

In [ ]:
#сохраняем размеры стихотворений в файл
df = pd.read_csv('poem_table.csv', encoding='utf-8')
df['ритм'] = score_line
df.to_csv('poem_table.csv', index=False, encoding='utf-8')

In [ ]:
#сохраняем виды рифм стихотворений в файл
df = pd.read_csv('poem_table.csv', encoding='utf-8')
df['рифма'] = rhyme_line
df.to_csv('poem_table.csv', index=False, encoding='utf-8')

Кластеризация текстов

In [ ]:
#добавляем в таблицу с информацией о стихотворениях их тексты
text_list = datum ['poem_list']
df = pd.read_csv('poems_table.csv')
df['текст'] = text_list
df.to_csv('poems_table.csv', index=False, encoding='utf-8')

In [4]:
df = pd.read_csv('poem_table.csv')

In [ ]:
#рассчёт оптимального количества кластеров методом локтя
hv = HybridVectorizer( #настройка модели
    column_encodings={
        'текст': 'text',
        'тема': 'categorical',
        'ритм': 'categorical',
        'рифма': 'categorical'
    }
)
vectors = hv.fit_transform(df)

inert = []
K_range = range(2, 10)  #задаём диапазон количества кластеров (были опробованы 2-20 и 2-10)
for k in K_range:
    print(f"k={k}...")
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(vectors)
    inert.append(kmeans.inertia_)

plt.figure(figsize=(8,5)) #настраиваем и строи график
plt.plot(K_range, inert, marker='o')
plt.xlabel('Кластеры')
plt.ylabel('Инерция')
plt.grid(True)
plt.show()

In [5]:
#кластеризируем тексты
hv = HybridVectorizer(
    column_encodings={
        'текст': 'text',
        'тема': 'categorical',
        'ритм': 'categorical',
        'рифма': 'categorical'
    },
)
vectors = hv.fit_transform(df)
n_clusters = 7
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)

2026-02-24 11:04:15,839 [INFO] HybridVectorizer - Using CPU for text encoding (GPU not available)
2026-02-24 11:04:15,842 [INFO] HybridVectorizer - Initializing HybridVectorizer
2026-02-24 11:04:15,845 [INFO] HybridVectorizer - Loading text model: all-MiniLM-L6-v2
2026-02-24 11:04:15,855 [INFO] sentence_transformers.SentenceTransformer - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2026-02-24 11:04:17,058 [INFO] httpx - HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json "HTTP/1.1 307 Temporary Redirect"
2026-02-24 11:04:17,088 [INFO] httpx - HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/modules.json "HTTP/1.1 200 OK"
2026-02-24 11:04:17,257 [INFO] httpx - HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json "HTTP/1.1 307 Temporary Redirect"
2026-02-

🔄 Processing data
📝 Encoding text: текст...


2026-02-24 11:05:14,342 [INFO] HybridVectorizer - [кластеры] -> numerical (dtype: int64)


✅ Generated 480 vectors with 395 dimensions


In [ ]:
#сохраняем номера кластеров текстов в файле
df['кластеры'] = kmeans.fit_predict(vectors)
df.to_csv('poem_table.csv', index=False, encoding='utf-8')

In [ ]:
def find_closest_in_cluster(poem_index): #создаём функцию для поиска текста, наиболее схожего с исходным
    cluster = df.loc[poem_index, 'кластер']
    target_vec = vectors[poem_index]
    mask = (df['hybrid_cluster'] == cluster) & (df.index != poem_index) #находим индексы стихотворений одного кластера, исключая исходный текст
    candidate_indices = df.index[mask].tolist()
    candidate_vectors = vectors[candidate_indices]
    distances = np.linalg.norm(candidate_vectors - target_vec, axis=1) #вычисляем расстояния между векторами
    best_idx_local = np.argmin(distances) #находим вектор с наименьшим расстоянием до исходного
    best_idx = candidate_indices[best_idx_local]
    return (best_idx)

Бот и анализ сходства текстов при помощи результатов кластеризации

In [ ]:
titles = datum['title_list'][0]
authors = datum ['author_list'][0]
bot = TeleBot("XXX")  #здесь содержится уникальный код бота

with open ('poem_data.json', 'r', encoding='utf-8') as file: #открываем созданный ранее файл с данными, полученными в ходе парсинга
    datum = json.load(file)

@bot.message_handler(commands=['start']) #создаём первое сообщение, отправляемое ботом после запуска
def start(message):
    bot.send_message(message.from_user.id, "Здравствуй! Я - твой личный проводник в мир поэзии. \n\nВведи название последнего понравившегося тебе стихотворения и получи персональную подборку текстов для чтения.")
    bot.register_next_step_handler(message, get_poem_name)

def get_poem_name(message): #запускаем процесс поиска стихотворений
    global poem_name
    poem_name = message.text.strip()
    bot.send_message(message.from_user.id, 'Вот пять текстов, которые я могу тебе порекомендовать:')
    give_texts (message)

def give_texts (message):
    x = poem_finder (poem_name, message) #при помощи полученного названия находим стихотворение в корпусе
    y = comparing_texts (x, vectors) #находим тексты, наиболее схожие с заданным
    recommended_author = authors[y]
    recommended_title = ' '.join(titles[y])
    bot.send_message (
        message.chat.id,
        f"Вот что я могу вам порекомендовать:\n"
        f"\"{recommended_title}\"\n{recommended_author}",
        parse_mode='HTML'
    ) #отправляем пользователю найденный текст

def poem_finder (poem_name, message):
        poem_name = set(word_tokenize (poem_name))
        c = 0
        for i in (datum ['title_list'][0]): #находим название, наиболее похожее на заданное
            m = set (i)
            k = jaccard_similarity (poem_name, m)
            if k > c:
                c = k
            else:
                number = (datum ['title_list'][0]).index (i)
        return (number)
        
def jaccard_similarity(set_a, set_b): 
    intersection = len(set_a & set_b)   
    union = len(set_a | set_b)          
    if union == 0:
        return 0                        
    return (intersection / union)
            
def comparing_texts (poem_index, vectors): #ищем текст, наиболее схожего с исходным
    cluster = df.loc[poem_index, 'кластеры']
    target_vec = vectors[poem_index]
    mask = (df['кластеры'] == cluster) & (df.index != poem_index) #находим индексы стихотворений одного кластера, исключая исходный текст
    candidate_indices = df.index[mask].tolist()
    candidate_vectors = vectors[candidate_indices]
    distances = np.linalg.norm(candidate_vectors - target_vec, axis=1) #вычисляем расстояния между векторами
    best_idx_local = np.argmin(distances) #находим вектор с наименьшим расстоянием до исходного
    best_idx = candidate_indices[best_idx_local]
    return (best_idx)

#запускаем приём сообщений ботом
bot.polling(none_stop=True)